In [2]:
#keras Functional Model
import keras

from keras.layers import Input , Dense
from keras.models import Model

import keras.backend as K

In [2]:
inputs = Input(shape=(784 , ))

In [3]:
x = Dense(64 , activation='relu')(inputs)
x = Dense(64 , activation='relu')(x)

predictions = Dense(10 , activation='softmax')(x)

In [4]:
model = Model(inputs=inputs , outputs=predictions)

model.compile(optimizer='rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(data , labels)

NameError: name 'data' is not defined

In [6]:
#模型类似函数 可以作为运算的操作
will_input_data = Input(shape=(784 , ))
y = model(will_input_data)
#模型可以被作为层一样
#调用一个模型时 即重用结构 也重用权重

In [7]:
from keras.layers import TimeDistributed

input_sequences = Input(shape=(20,784))

processed_sequences = TimeDistributed(model)(input_sequences)

In [ ]:
#http://keras-cn.readthedocs.io/en/latest/getting_started/functional_API/

http://keras-cn.readthedocs.io/en/ltest/images/multi-input-multi-output-graph.png

In [4]:
#multiple input and output
from keras.layers import Input , Embedding , LSTM , Dense
from keras.models import Model
import keras

#100 one-hot vectors(10000-dimensional)
main_input = Input(shape=(100,) , dtype='int32' ,
                  name = 'main_input')

#one_hot to 512-dimensional vectors (embedding vector)
#word2vec
x = Embedding(output_dim = 512 , input_dim = 10000,
             input_length = 100)(main_input)

#transform the vector sequence into a single vector(32-dimensional)
lstm_out = LSTM(32)(x)


#extra loss (for smoothly train LSTM Embedding layer)
auxiliary_output = Dense(1 , activation='sigmoid',
                        name='aux_output')(lstm_out)


#extra input
#auxiliary_input is unknown
auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out , auxiliary_input])

x = Dense(64 , activation='relu')(x)
x = Dense(64 , activation='relu')(x)
x = Dense(64 , activation='relu')(x)

#main loss
main_output = Dense(1 , activation='sigmoid' ,
                   name='main_output')(x)

model = Model(inputs=[main_input , auxiliary_input],
             outputs=[main_output , auxiliary_output])


#compile method 1
model.compile(optimizer='rmsprop',
             loss=['binary_crossentropy','binary_crossentropy'],
             loss_weights=[1.0 , 0.2])

model.fit([headline_data , additional_data] , [labels , labels] , epochs=50,
         batch_size=32)

#compile moethod 2
#前提是必须人为指定name参数
model.compile(optimizer='rmsprop',
             loss={'main_output':'binary_crossentropy',
                   'aux_output':'binary_crossentropy'},
             loss_weights={'main_output':1.0,
                           'aux_output':0.2})

model.fit({'main_input':headline_data,
           'aux_input':additional_data},
         {'main_output':labels,'aux_outpus':labels},
         epoch=50 , batch_size=32)

NameError: name 'headline_data' is not defined

In [ ]:
#shared layer
import keras
from keras.layers import Input , LSTM , Dense
from keras.models import Model

#140 one-hot vectors(256-dimensional)
tweet_a = Input(shape=(140 , 256))
tweet_b = Input(shape=(140 , 256))

#shared lstm layer
shared_lstm = LSTM(64)

encoded_a = shared_lstm(tweet_a)
encoded_b = shared_lstm(tweet_b)

#concat them
merged_vector = keras.layers.concatenate([encoded_a , encoded_b],
                                        axis=-1)

predictions = Dense(1 , activation='sigmoid')(merged_vector)

model = Model(inputs=[tweet_a  ,tweet_b] , outputs=predictions)

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.fit([data_a , data_b] , labels , epochs=10)

In [ ]:
a = Input(shape=(140,256))

#output 32-dimensional vectors
lstm = LSTM(32)

encoded_a = lstm(a)

assert lstm.output == encoded_a

#lstm.output 层的输出
#lstm.output_shape 层输出的shape

In [ ]:
a = Input(shape=(140 , 256))
b = Input(shape=(140 , 256))

lstm = LSTM(32)
encoded_a = lstm(a)
encoded_b = lstm(b)

#lstm.output error

assert lstm.get_output_at(0) == encoded_a
assert lstm.get_output_at(1) == encoded_b


In [ ]:
from keras.layers import Conv2D
a = Input(shape=(32,32,3))
b = Input(shape=(64,64,3))

#one input
conv = Conv2D(16 , (3,3) , padding='same')
conved_a = conv(a)

assert conv.input_shape == (None , 32 , 32 , 3)

#multiple input
conved_b = conv(b)

assert conv.get_input_shape_at(0) == (None , 32 , 32 , 3)
assert conv.get_input_shape_at(1) == (None , 64 , 64 , 3)


In [ ]:
#google inception model

from keras.layers import Conv2D , MaxPooling2D , Input

input_img = Input(shape=(256,256,3))

tower_1 = Conv2D(64 , (1,1) , padding='same',
                activation='relu')(input_img)
tower_1 = Conv2D(64 , (3,3) , padding='same',
                activation='relu')(tower_1)

tower_2 = Conv2D(64 , (1,1) , padding='same',
                activation='relu')(input_img)
tower_1 = Conv2D(64 , (5,5) , padding='same',
                activation='relu')(tower_2)

tower_3 = MaxPooling2D((3,3) , strides=(1,1) , padding='same')(input_img)
tower_3 = Conv2D(64 , (1,1) , padding='same',
                 activation='relu')(tower_3)

output = keras.layers.concatenate([tower_1 , tower_2 , tower_3] , axis=1)

In [ ]:
#residual netword
from keras.layers import Conv2D , Input

x = Input(shape=(256,256 , 3))

y = Conv2D(3 , (3,3) , padding='same')(x)

z = keras.layers.add([x,y])

In [5]:
from keras.layers import Conv2D , MaxPooling2D , Input , Dense , Flatten
from keras.models import Model
import keras
#使用两个Model
#MNIST
#判断两个MNIST图片是否是相同的数字
digit_input = Input(shape=(27,27,1))
x = Conv2D(64 , (3,3))(digit_input)
x = Conv2D(64 , (3,3))(x)
x = MaxPooling2D((2,2))(x)

out = Flatten()(x)

vision_model = Model(digit_input , out)


digit_a = Input(shape=(27,27,1))
digit_b = Input(shape=(27,27,1))

out_a = vision_model(digit_a)
out_b = vision_model(digit_b)

concatenated = keras.layers.concatenate([out_a , out_b])
out = Dense(1 , activation='sigmoid')(concatenated)

classification_model = Model([digit_a , digit_b] , out)

In [7]:
#对图片进行提问
from keras.layers import Conv2D , MaxPooling2D , Flatten
from keras.layers import Input , LSTM , Embedding , Dense
from keras.models import Model , Sequential

vision_model = Sequential()
vision_model.add(Conv2D(64,(3,3) , activation='relu',
                       padding='same',
                       input_shape(224,224,3)))
vision_model.add(Conv2D(64 , (3,3),activation='relu'))
vision_model.add(MaxPooling2D((2,2)))

vision_model.add(Conv2D(128 , (3,3) , activation='relu',padding='same'))
vision_model.add(Conv2D(128 , (3,3) , activation='relu'))
vision_model.add(MaxPooling2D(2,2))

vision_model.add(Conv2D(256 , (3,3) , activation='relu',padding='same'))
vision_model.add(Conv2D(256 , (3,3) , activation='relu'))
vision_model.add(Conv2D(256 , (3,3) , activation='relu'))
vision_model.add(MaxPooling2D(2,2))

vision_model.add(Flatten())


image_input = Input(shape=(224,224,3))
encoded_image = vision_model(image_input)


question_input = Input(shape=(100 , ) , dtype='int32')
embedded_question = Embedding(input_dim=10000,
                             output_dim=256,
                             input_length=100)(question_input)
encoded_question = LSTM(256)(embedded_question)


merged = keras.layers.concatenate([encoded_question , encoded_image])

output = Dense(1000 , activation='softmax')(merged)

vqa_model = Model(inputs=[image_input , question_input] , outputs=output)

vqa_model.fit()

In [ ]:
#对视频进行提问
from keras.layers import TimeDistributed
#100帧的视频数据
video_input = Input(shape=(100 , 224,224,3))

encoded_frame_sequencec = TimeDistributed(vision_model)(video_input)
encoded_video = LSTM(256)(encoded_frame_sequencec)

question_encoder = Model(inputs=question_input , outputs=encoded_question)

video_question_input = Input(shape=(100 , ) , dtype = 'int32')
encoded_video_question = question_encoder(video_question_input)


merged = keras.layers.concatenate([encoded_video , encoded_video_question])
output = Dense(1000 , activation='softmax')(merged)

vodel_qa_model = Model(inputs=[video_input , video_question_input],
                      outputs=output)

In [8]:
import keras.backend as K

In [9]:
K.image_data_format()

#(sample_size , high , width , channel)

'channels_last'

In [11]:
from keras.models import load_model #load from h5 file
from keras.models import model_from_json #load from json file
from keras.models import model_from_yaml #load from yaml file
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=10 , input_dim = 784))

#会把模型的所有东西都保存
#optimizer的状态都会保存
model.save('my_model.h5')

#load it
model = load_model('my_model.h5')

In [ ]:
#只保存模型结构
json = model.to_json()
model = model_from_json(json)

yaml = model.to_yaml()
model = model_from_yaml(yaml)

In [ ]:
#保存模型权重
model.save_weights('xx.h5')
model.load_weights('xx.h5') #完全相同的结构

#通过name参数 加载模型
model.load_weights('xx.h5' , by_name=True)

In [ ]:
#获取中间层的输出
#method 1
from keras.models import Model
#构造model
layer_name = '指定的layer name'
intermediate_layer_model = Model(inputs = model.input,
                                outputs=model.get_layer(layer_name).output)

intermediate_output = intermediate_layer_model.predict(input_data)

#method 2
import keras.backend as K

get_3rd_layer_output = K.function([model.layers[0].input],
                              [model.layers[3].output]) 

layer_output = get_3rd_layer_output([X])[0]

#method 3
get_3rd_layer_output = K.function([model.layers[0].input , K.learning_phase()],
                                  [model.layers[3].output])

#测试模式下查看第三层输出
layer_output = get_3rd_layer_output([X , 0])[0]
#训练模型下查看第三层输出
layer_output = get_3rd_layer_output([X , 1])[0]

In [15]:
#验证集损失不再下降 中断训练
#callback 机制
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss' , patience = 2)

#5-fold validation
#先分出验证数据 在shuffle
#所以验证数据 一定是最后的20%
model.fit(X , y , validation_split=0.2 , callbacks=[early_stopping])

AttributeError: 'Dense' object has no attribute 'inputs'

In [ ]:
#stateful LSTM
model = Sequential()
model.add(LSTM(32 , input_shape=(10,16),batch_size=32 , stateful=True))
model.add(Dense(16 , activation='softmax'))

model.compile(optimizer='rmsprop' , loss = 'categorical_crossentropy')

#用一个batch更新一次梯度
model.train_on_batch(X[: , :10 , :] , np.reshape(X[: , 10 , :] , (32,16)))

#再用一个batch更新一次梯度
model.train_on_batch(X[: , 10:20 , :] , np.reshape(X[: , 20 , :] , (32,16)))

#重置网络所有层
model.reset_states()
#重置指定层
model.layers[0].reset_states()

In [16]:
#冻结层 就是不参与训练 权重不能变
Dense(units=32 , trainable = False) #冻住了

In [ ]:
x = Input(shape=(32 , ))
layer = Dense(32)
layer.trainable = False
y = layer(x)

#不会更新权重
frozen_model = Model(x , y)
frozen_model.compile(optimizer = 'rmsprop' , loss='mse')

#会更新了
layer.trainable = True
trainable_model = Model(x , y)

trainable_model.compile(optimizer='rmsprop' , loss='mse')

frozen_model.fit(data , labels)

trainable_model.fit(data , labels)

In [ ]:
model = Sequential()
model.add(Dense(32 , activation='relu' , input_dim=784))
model.add(Dense(32 , activation='relu'))

print(len(model.layers)) #2

#类似栈 删除一层
model.pop()
print(len(model.layers)) #1

In [ ]:
#使用已经预训练的模型
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3

model = VGG16(weights = 'imagenet' , include_top = True) #需要下载

In [ ]:
#使用hdf5数据
import h5py

with h5py.File('input/file.hdf5' , r) as f:
    X_data = f['X_data']
    model.predict(X_data)

In [ ]:
#keras 配置文件
%USERPROFILE%/.keras/keras.json
#缓存数据集文件
%USERPROFILE%/.keras/datasets/

In [7]:
from keras.models import Sequential

model = Sequential()
model.add(Dense(32 , activation='relu' , input_dim=784))
model.add(Dense(32 , activation='relu'))

In [8]:
model.get_config() #查看层的详细信息


[{'class_name': 'Dense',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'batch_input_shape': (None, 784),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'dtype': 'float32',
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'name': 'dense_1',
   'trainable': True,
   'units': 32,
   'use_bias': True}},
 {'class_name': 'Dense',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_r

In [ ]:
model_config = model.get_config() #保存配置信息
#可以使用config文件 用两种方式进行构建模型
model = Model.from_config(model_config)

model = Sequential.from_config(model_config)